# Today's challenge: a sentiment analysis of Trump tweets

The goal for today is to introduce Python programming via a real world text analysis task: conducting a sentiment analysis of Twitter data. Our learning objectives include:

1. Introducing Python via a real-world text analysis task.
2. Illustrating how to load data into (and write data out of) Python
3. Learning to implement "standard" text pre-processing in Python
4. Learning how to conduct a lexicon-based sentiment analysis

To achieve these objectives, we will focus on a dataset which contains all of Donald Trump's tweets for the year 2017. Our challenge is to build a simple lexicon (or dictionary) to track negative sentiment in Trump's Twitter feed. Let's get started!

#### Note on programming in Python

This notebook offers an introduction to progamming in the Python language. It's impossible to cover it all in a single notebook (or a single class!); however, this notebook highlights core aspects of Python that are important for this class. I highly recommend the (free and online!) book <a href=https://python.swaroopch.com/><i>A Byte of Python</i></a> if you would like to further study the ideas outlined in this notebook.

## Step 1: Load the required libraries

The first step in constructing our sentiment analysis script is load any required libraries using the `import` statement: 

In [2]:
import os # Needed to change your working directory
os.chdir('/Users/tcoan/git_repos/notebooks') # CHANGE THIS DIRECTORY!
import json # Needed to load the trump_tweets_2017.json file

# We will use a couple of regular expressions in this tutorial.
# I have another notebook that provides more details on working with
# regular expressions.
import re

# Import NLTK
import nltk
nltk.download('punkt')     # Download NLTK "data" once (see
nltk.download('stopwords') # https://www.nltk.org/data.html)

# Tokenization
from nltk.tokenize import sent_tokenize

from nltk.tokenize import RegexpTokenizer

# Stopwords
from nltk.corpus import stopwords

# Stemming and lemmatization
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Demonstrate pre-built sentiment lexicons
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Import the pandas library using the namespace "pd" to 
# save on typing. We will use pandas for reading/writing data,
# as well as for the occasional descriptive statistic
import pandas as pd

[nltk_data] Downloading package punkt to /Users/tcoan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tcoan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Note that if there is a library that you need to use that is not pre-installed with Anaconda, you can install it within Jupyter using the following:

In [ ]:
%pip install spacy

## Step 2: Read text data into Python for analysis

The next step is to read in tweets for our sentiment analysis. The trump_tweets_2017.json is a <a href="https://www.w3schools.com/js/js_json_intro.asp">JSON</a> formatted file and so we need to use the `json` library to open it:

In [ ]:
# The "opens" a connection to the trump_tweets data on disk and "loads"
# into an object called "tweets"
with open('data/trump_tweets_2017.json', 'r', encoding="utf-8") as jfile:
    tweets = json.load(jfile)

Great, so we've "loaded" our JSON file, but what is actually stored in <b>variable</b> `tweets`? We could print it, but that doesn't help all that much!

In [ ]:
tweets[0]['source']

It turns out that JSON files are read as a list of dictionaries.

### Reading and writing data with `pandas`

We use the `json` library to read our Trump data, but what if your data is saved in something other than JSON? Python has standalone libraries for reading files stored in different formats (e.g., the <a href=https://docs.python.org/3/library/csv.html>csv</a> module). However, the `pandas` library offers a convienent approach to reading and writing files in pretty much any format out there.

Note that `pandas` offers so, so much more than just reading and writing files. It's provides an R-like environment for data wrangling and analysis, and is quickly becoming the "go to" library for data analysis in Python. For an introduction to `pandas`, please check out:

<https://www.learndatasci.com/tutorials/python-pandas-tutorial-complete-introduction-for-beginners/>

However, we won't spend too much time on `pandas` in this section of the course, but instead use it for reading/writing data, as well as a bit of descriptive statistics.

Reading data in `pandas` is as easy as:

In [ ]:
# Read the Trump tweets CSV into a pandas "dataframe"
tweets_df = pd.read_csv('data/trump_tweets_2017.csv')

The `tweets_df` object is a `pandas` dataframe, which operates similarly to an R data frame:

In [ ]:
tweets_df.head()

In [ ]:
tweets_df['retweet_count'].mean()

To convert our `trump_df` to list of dictionaries as above, we need to run the following code:

In [ ]:
tweets = tweets_df.to_dict('records')
tweets[0]

<span style="color: red;">WARNING</span>: The code written below to clean and process our text data assumes that our data is a list of dictionaries. If you try to use this code directly on a `pandas` dataframe it will not work. If using `pandas` to read data for this class, make sure to add `to_dict('records')` when loading, e.g.:

In [3]:
tweets = pd.read_csv('data/trump_tweets_2017.csv').to_dict('records')

## Step 3: prepare text for analysis (aka pre-processing)

With our list of tweets in hand, we are now need to get out text ready for analysis. While the process of "pre-processing" text will vary a bit based on the specific analysis employed, we will cover the so-called "standard" pre-processing procedures:

1. Tokenization
2. Convert to lower (or upper) case
3. Punctuation removal
4. Stopword removal

Other common pre-processing procedures **not** covered in this notebook include:

5. Expanding contractions
6. Lemmatizing or stemming
7. Removing numbers

I will cover these additional pre-processing procedures in a seperate notebook (extended-preprocessing.ipynb) for anyone that is interested.

Great, we are ready to get going. Before processing our text, however, we need to get a sense of how Python understands `strings`.

### Strings and string methods

Python has all of the standard data types (number, strings, bool), but `strings` (or text!) is obviously quite important for text analysis class.

In [ ]:
tweet = tweets[0]['text']
tweet

Strings are "iterable", meaning that they have an index:

In [ ]:
len(tweet)

They also have a set of "methods" (or functions) that will be useful for us throughout the course. Here are some of the most important.

`lower()` and `upper()`:

In [ ]:
'This text NEEDS to be lowercase.'.lower()

`strip()` white space from the ends of a string:

In [ ]:
'   We need to strip the white space off of this sentence.         '.strip()

`split()` a string:

In [ ]:
tweet.split(' ')

I could go on and on here. The point is do yourself a favor and review the following methods:

<a href="https://www.w3schools.com/python/python_ref_string.asp">https://www.w3schools.com/python/python_ref_string.asp</a>

### Tokenization

Tokenization is just the process of splitting up our strings into smaller parts (usually sentences or words). We've already seen one way to do this using the `split()` method:

In [ ]:
toks = tweet.split(' ')
print(toks)

#### `NLTK` for tokenization (fast, but less accurate)

There are also times when you want to first tokenize a string into sentences and then into words. Here, we can use the `sent_tokenize` function that we imported above to do the job:

In [ ]:
sentences = sent_tokenize(tweet)
print(sentences)

For our simple twitter sentiment analysis, we will ignore the sentence structure and treat our text as a <b>bag of words</b>. We'll also use the `word_tokenize` function from `nltk` to get a cleaner set of tokens than what was produced using `split()`:

In [ ]:
toks = word_tokenize(tweet)
print(toks)

Nice, so now we know how to tokenize a single tweet. However, we have 2,275 tweets to process. <b>Soluton</b>: the trusty old `for` loop.

In [ ]:
this_dict = f'This is {len(tweets)}'

#### `spacy` for tokenization (slow, but accurate)

In terms of accuracy, `spacy` is by far the best approach to tokenization in Python. To use `spacy`, we need to `import` the library and load one of the pre-trained `spacey` models:

In [4]:
import spacy
# Load one of the spacy models
nlp = spacy.load('en_core_web_sm')

You would then tokenize a text into sentences using the following:

In [ ]:
text = "The outrageously corrupt fake news media is lying to you about this election. The Democrats attempted the most brazen and outrageous election theft in history. We need to stop the steal!"
parsed = nlp(text) # parsed is a spacy object with tons of goodness included!
sents = list(parsed.sents)

In [ ]:
# Let's take a look at the first sentence
print(sents[0])

We can tokenize our sentences into words as follows:

In [ ]:
toks = list(sents[0])
print(toks)

In [ ]:
toks[1].head

One thing to note when working with `spacy` is that even though the above `toks` list looks like a string when printed, it's actually a `spacy` **object** which has 

In [ ]:
# What's the second word in the first sentence?
print(toks[1])
# What's the part of speech (POS) of the second word in the first sentence?
print(toks[1].pos_)
# What's the dependency?
print(toks[1].dep_)
# What adjective is the word modifying?
print(toks[1].head)

If we want the text strings, then we need to extract them from the `spacy` object using a loop:

In [ ]:
tokens = []
for tok in toks:
    tokens.append(tok.text)

# Let's take a look:
print(tokens)

And we can confirm that these are now strings:

In [ ]:
print(type(tokens[0]))

Note that we can actually write this code a bit more efficiently by using what is called "<b>list comprehension</b>" in Python:

In [ ]:
tokens_ = [tok.text for tok in toks]
print(tokens_)

# We can confirm that the list comprehension results are the same as the loop:
print(tokens_ == tokens)

One last thing to note is that if you don't care about tokenizing into sentences and just want the words, you can directly work with the `parsed` object created above:

In [ ]:
toks = [tok.text for tok in parsed]
print(toks)

### Tokenizing and normalizing an entire corpus ( `for` loop)

Once we understand how to tokenize a single text, then tokenizing an entire corpus of tweets is easy -- we just **loop** over each tweet and tokenize. It's also a good idea to do some additional **normalization** (i.e., removing punctuation, converting to lowercase) looping over our tweets. For instance, using our `tweets` data loaded above and assuming that we wanted to use `spacy` for tokenizations, we could use the following "clunky" loop:

In [ ]:
tokens = []
for tweet in tweets:
    # Parse the tweet text using spacy
    parsed = nlp(tweet['text'])
    tweet_tokens = [] # Preallocate a list to hold the tokens for each tweet
    for tok in parsed:
        # If the token is not punctuation, save it
        if tok.pos_ != 'PUNCT':
            # Save the lowercase version of the string
            tweet_tokens.append(tok.text.lower())
    # Save the tweet tokens and move to the next
    tokens.append(tweet_tokens)    

In [ ]:
# Let's view the first tweet to see if this worked
print(tokens[0])

Note that the "clunky" loop is fine, but that's not how I would have done this! I would have used something like:

In [5]:
# Define a function that tokenize and normalize
def tokenize(text):
    '''
       Takes a text string, tokenizes the text using spacy,
       removes punctuation, and converts to lowercase.
    '''
    return [tok.text.lower() for tok in nlp(text) if tok.pos_ != 'PUNCT']

# Use list comprehension to loop over the tweets and process
tokens = [tokenize(tweet['text']) for tweet in tweets]

### <span style="color: green;">Activity 1</span> : tokenizing tweets using `NLTK`

We've seen how to tokenize and normalize using `spacy`. However, to test your knowledge, let's see if you can carry out these tasks using `NLTK` instead. Before you get going, let's take a quick look at how the `word_tokenize()` function deals with punctuation

Our use of the `word_tokenize()` function makes removing punctuation very, very easy. Check out how `word_tokenize()` handles punctuation:

In [ ]:
print(word_tokenize('I love this class!!!!!'))

As such, all we need to do is remove strings with a length == 1.

Now to the the activity!

1. **Tokenize**: Loop over each "tweet" in our `tweets` data and apply the `word_tokenize` function
2. **Remove punctuation**: Only keep tokens with a length greater than 1.
3. **Lowercase**: Return each the lower case of each token.

In [ ]:
word_tokenize(tweets[0]['text'])

In [ ]:
tokens_nltk = []
for tweet in tweets:
    tokens_nltk.append([tok.lower() for tok in word_tokenize(tweet['text']) if len(tok) > 1])

In [ ]:
tokens_nltk[0]

### Removing stopwords

In many analyses, it makes sense to remove so-called **stopwords**. These are words that show up frequently in a text, but add very little meaning.

In [6]:
# Pull in NLTK's English language stopword list.
stops = stopwords.words('english')
print(stops[0:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [9]:
text = ['travis', 'travis', 'riley', 'dre']
print(list(set(text)))

['travis', 'dre', 'riley']


First thing to notice is our stopword list assumes lowercase. You always need to check this! But how do we remove these words from our list? Let's to it the long way first and then we will use the equivalent list comprehension:

In [10]:
tokens_no_stops = [] # preallocate a list to hold tweet-level data
# First loop is over tokenized tweets
for toks in tokens:
    toks_no_stops = [] # preallocate a list to hold token-level data
    # Next loop over the individual words in each tokenized tweet
    for tok in toks:
        # Check if the word (tok) is in the stopword list.
        # Need to add .lower()!!!
        if tok.lower() not in set(stops):
            toks_no_stops.append(tok)
    # Store result and move to next tweet
    tokens_no_stops.append(toks_no_stops)

In [11]:
print('Tokens with stopwords removed: ', tokens_no_stops[0], '\n')
print('Original tokens: ', tokens[0])

Tokens with stopwords removed:  ['jobs', 'kicking', 'companies', 'coming', 'back', 'u.s.', 'unnecessary', 'regulations', 'high', 'taxes', 'dramatically', 'cut', 'get', 'better', 'much', 'come'] 

Original tokens:  ['jobs', 'are', 'kicking', 'in', 'and', 'companies', 'are', 'coming', 'back', 'to', 'the', 'u.s.', 'unnecessary', 'regulations', 'and', 'high', 'taxes', 'are', 'being', 'dramatically', 'cut', 'and', 'it', 'will', 'only', 'get', 'better', 'much', 'more', 'to', 'come']


Here's a more compact version using list comprehension:

In [12]:
tokens_no_stops = [] # preallocate a list to hold tweet-level data
for toks in tokens:
    tokens_no_stops.append([tok for tok in toks if tok not in set(stops)])

Note that there is still some junk in this tweet (e.g., 'amp', 'https', etc.). It's not uncommon to have "corpus-specific" (or "extended") stopwords. We can add additional words to the stopword list:

In [13]:
stops_to_add = ['amp', 'https', 'co']

# Concatenate the two lists
extended_stops = stops + stops_to_add

# The last stopword is now 'co'
print(extended_stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

We just need to use this extended stop word list and we are good to go:

In [14]:
tokens_no_stops = [] # preallocate a list to hold tweet-level data
for toks in tokens:
    tokens_no_stops.append([tok for tok in toks if tok not in set(extended_stops)])

### Step 4: Build and use your lexicon (or dictionary)

What words should go into our lexicon of "negative" words? While there are a number of words with clear, negative conotations, lexicon-based approaches often need to be tailored to a specific task (or at least validated for the specific task at hand). We will start by illustrating how to build and employ our own lexicon -- as this could be useful across a range of tasks, not just sentiment analysis -- and then I will quickly demonstate how to use existing sentiment lexicons in Python.

#### Starting simple: looking up a single word

Let's start simple and examine the presence of absence of a single word in our corpus: fake. Here's one way of completing this task:

In [15]:
fake = []
for toks in tokens_no_stops:
    if 'fake' in toks:
        fake.append(1)
    else:
        fake.append(0)

In [17]:
fake[0:10]

[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]

In [ ]:
'fake' in ' '.join(tokens_no_stops[1])

#### Building our negative sentiment lexicon

A lexicon (or dictonary) is just a list of words. That's it! So we can start by created list of "negative" words:

In [18]:
negative = ['fake', 'hoax', 'idiot', 'moron', 'phony', 'fight', 'dishonest', 'unfair']

Done! Okay, okay, this list isn't exhaustive, but it is good enough to show you how to implement your own lexicon-based approach. Using our `negative` words lexicon takes a little code, but we have all of the tools to implement it:

In [19]:
# Start by looping over each list of tokens
negative_sentiment_score = []
for toks in tokens_no_stops:
    # Define a counter to hold the number of negative words
    negative_words = 0
    # It is possible for a tweet not to have text. If this is the case,
    # we need to skip it.
    if len(toks) == 0:
        print("This tweet does not have text. Setting negative sentiment to 0.")
        negative_sentiment_score.append(0) 
    else:
        # Then loop over each token in each list
        for tok in toks:
            if tok in set(negative):
                negative_words += 1
        # Add normalized sentiment score
        negative_sentiment_score.append(negative_words/len(toks)) 

This tweet does not have text. Setting negative sentiment to 0.


In [24]:
# Start by looping over each list of tokens
negative_sentiment_score = []
for i,toks in enumerate(tokens_no_stops):
    print(i)
    # Define a counter to hold the number of negative words
    negative_words = 0
    # Then loop over each token in each list
    for tok in toks:
        if tok in set(negative):
            negative_words += 1
    # Add normalized sentiment score
    negative_sentiment_score.append(negative_words/len(toks))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ZeroDivisionError: division by zero

In [ ]:
negative_sentiment_score[0:10]

## Using pre-built sentiment lexicons

There are a number of good sentiment lexicons already available and `NLTK` provides access to a number of these (see the <a href="https://www.nltk.org/api/nltk.sentiment.html">nltk.sentiment api documentation</a> for a full list). Once lexicon that has been shown to work well across a range of tasks is <a href="http://scholar.google.co.uk/scholar_url?url=https://ojs.aaai.org/index.php/ICWSM/article/download/14550/14399&hl=en&sa=X&ei=rUoRYKXUDIfCmgGXoLToAw&scisig=AAGBfm22NmYm4wMvPAvkQZuGz-24V1Wu1A&nossl=1&oi=scholarr">VADER</a>. Here's how you can use VADER in `NLTK`. First, start by instantiating the `SentimentIntensityAnalyzer` class:

In [25]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tcoan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [26]:
vader = SentimentIntensityAnalyzer()

In [27]:
print(vader)

In [31]:
vader.polarity_scores("This class sucks!!!")

{'neg': 0.628, 'neu': 0.372, 'pos': 0.0, 'compound': -0.5229}

Note that there is no pre-processing necessary -- you can simply pass your text in "raw" form. Applying to our Trump example:

In [33]:
for tweet in tweets:
    # Here I'm adding a new field to the "tweets" dictionary directly
    # While this changes the original dictionry (and so requires caution),
    # it ensures that we have all of the relevant meta-data about the tweet
    # easily accessble.
    tweet['negative_sentiment'] = vader.polarity_scores(tweet['text'])['neg']

In [34]:
print(tweets[1])

{'source': 'Twitter for iPhone', 'id_str': 9.47e+17, 'text': 'I use Social Media not because I like to, but because it is the only way to fight a VERY dishonest and unfair “press,” now often referred to as Fake News Media. Phony and non-existent “sources” are being used more often than ever. Many stories &amp; reports a pure fiction!', 'is_retweet': False, 'retweet_count': 50342, 'favorite_count': 195754, 'negative_sentiment': 0.344}


In [ ]:
tweets_to_write = pd.DataFrame(tweets).to_csv('tweets_sentiment.csv')

### Bonus material: defining a pre-processing function using `NLTK`

It's good practice to roll up our various bits of pre-processing in a single **function**. Functions allow to reuse pieces (or blocks) of code. We do so by declaring a function using the `def` statement. We have already used several of Python's built-in functions earlier in this tutoral. For instance, we "called" the `len` function to get the number of characters in a string. Python, however, makes it super easy to define your own functions.

For example, we can combine the main pre-processing steps above into a single function as follows:

In [ ]:
def process_tweet(tweet, stops):
    '''
    Helper function to pre-process tweet text. It takes an individual tweet,
    tokenizes it, converts to lowers, and removes punctuation.
    
    Args:
        tweet (str): The (unprocessed) tweet text
        stops (list): Stopword list to use
    
    Returns:
        list of lists: Returns processed tokens
    '''
    # Convert the entire tweet to lowercase, tokenize, and remove stop words
    tokens = [token for token in word_tokenize(tweet.lower()) 
          if token not in set(stops)]
    # Remove punctuation
    tokens_no_punct = [token for token in tokens if len(token) > 1]
    # Remove numbers and return the processed list of tokens
    return tokens_no_punct

In [ ]:
help(process_tweet)

We can then process our corpus using a single line of code:

In [ ]:
tweets_clean = [process_tweet(tweet['text'], extended_stops) for tweet in tweets]

In [ ]:
print(tweets_clean[0])

We can set up a function to use our lexicon in the same exact way:

In [ ]:
def calculate_lexicon (tokens, lexicon):
    '''
    Takes a tokenized set of texts and counts the number of tokens
    in each text included in a lexicon.
    
    Args:
        tokens (list of list): Tokenized text
        lexicon (list of str): List of tokens to lookup
    '''
    result = []
    for token in tokens:
        lexicon_words = 0
        if len(token) == 0:
            result.append(None) 
        else:
            # Then loop over each token in each list
            for tok in token:
                if tok in set(lexicon):
                    lexicon_words += 1
            result.append(lexicon_words/len(toks)) 
    return result

In [ ]:
res = calculate_lexicon(tweets_clean, negative)